# The `parameter` statement

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * A `parameter` statement defines a parameter that can be passed from command line
  * SoS determines type of parameters from default values or types
  * `--no-var` is used to specify `False` for boolean parameters

The `parameter` statement specifies variables that can be specified from command line (or magics such as `%run` from SoS Notebook).

The basic format of the statement is

```
parameter: name = value
```
where
* `name` is the name of the parameter
* `value` is either a default value, or a type (e.g. `str`, `int`). Parameters specified with types are required

Parameters are specified with double dash syntax from command line.

In [1]:
%run --var 10

parameter: var = 5
print(f'var is {var}')

INFO: Running default :

var is 10


INFO: Workflow default (ID=2fe25ec65c2d111b) is executed successfully with 1 completed step.

## Optional and required parameters

Parameters with default values are optional. The default value will be used if the parameter is left unspecified

In [2]:
parameter: var = 5
print(f'var is {var}')

var is 5


When a type is specified, a parameter will be required and an error will be raised if a required parameter is not specified

In [3]:
%env --expect-error
parameter: var = int
print(f'var is {var}')

ERROR: scratch_0 (id=-2602400190122744123) returns an error.

ExecuteError: [0]: Argument var of type int is required

In [4]:
%run --var 10

parameter: var = int
print(f'var is {var}')

INFO: Running default :

var is 10


INFO: Workflow default (ID=9796d4b606d8fe49) is executed successfully with 1 completed step.

If you have a parameter with underscore `_`, it can be specified with either underscore (`_`) or dash (`-`).

In [5]:
%run --input_files a.txt

parameter: input_files = list
print(input_files)

INFO: Running default :

['a.txt']


INFO: Workflow default (ID=66269fd4c44b040b) is executed successfully with 1 completed step.

In [6]:
%run --input-files a.txt b.txt

parameter: input_files = list
print(input_files)

INFO: Running default :

['a.txt', 'b.txt']


INFO: Workflow default (ID=11038fe1f38f0451) is executed successfully with 1 completed step.

## Type of parameters

The types of parameters are determined by the default value or type specification, and determines how they should be passed from command line.

### Simple Python types

SoS automatically determines the type of default values and convert your input data to the type. For example, the type of `cutoff` is determined to be an integer so it accepts an integer value from command line:

In [7]:
%run --cutoff 5
parameter: cutoff = 0

print(cutoff)

INFO: Running default :

5


INFO: Workflow default (ID=a3151114b3e2553f) is executed successfully with 1 completed step.

An error will be raised if you pass a string,

In [8]:
%env --expect-error

%run -v0 --cutoff zero
parameter: cutoff = 0

print(cutoff)

ERROR: ERROR step_executor.py:1743 - default (id=-7112044658713010777) returns an error. - None

[] Failed with 1 step processed (1 job completed)
ERROR: [default]: [0]: argument --cutoff: invalid int value: 'zero'


RuntimeError: Workflow exited with code 1

even a float value

In [9]:
%env --expect-error

%run -v0 --cutoff 5.1
parameter: cutoff = 0

print(cutoff)

ERROR: ERROR step_executor.py:1743 - default (id=-5162514331214072094) returns an error. - None

[] Failed with 1 step processed (1 job completed)
ERROR: [default]: [0]: argument --cutoff: invalid int value: '5.1'


RuntimeError: Workflow exited with code 1

If you intended to accept an float value, use a default value in float:

In [10]:
%run --cutoff 5.1
parameter: cutoff = 0.

print(cutoff)

INFO: Running default :

5.1


INFO: Workflow default (ID=335a5f509a9d19a3) is executed successfully with 1 completed step.

### Yes or no (Boolean types)

Boolean data types can be specified using `--opt` and `--no-opt`:

In [11]:
%run -v0 --test-mode

parameter: test_mode = bool
print(f'test_mode is {test_mode}')

[#] 1 step processed (1 job completed)


In [12]:
%run -v0 --no-test-mode

parameter: test_mode = bool
print(f'test_mode is {test_mode}')

[#] 1 step processed (1 job completed)


This works the same for boolean variables with default value:

In [13]:
%run -v0 

parameter: test_mode = False
print(f'test_mode is {test_mode}')

[#] 1 step processed (1 job completed)


In [14]:
%run -v0 --test-mode

parameter: test_mode = False
print(f'test_mode is {test_mode}')

[#] 1 step processed (1 job completed)


### List of strings

A list would be created if the parameter has a default value of type list. For example, a list `['A']` is returned because the default value has a list type.

In [15]:
%run --sample-names A
parameter: sample_names = []

print(sample_names)

INFO: Running default :

['A']


INFO: Workflow default (ID=082993c70fff6cb7) is executed successfully with 1 completed step.

SoS even understands the type of the values of the list and tries to follow it:

In [16]:
%run --values 4 5
parameter: values = [1, 2, 3]

print(values)

INFO: Running default :

[4, 5]


INFO: Workflow default (ID=900cb80ed5a077e6) is executed successfully with 1 completed step.

However, it is not yet possible to specify the type of values when you specify a required parameter of type list so all the values will be passed as strings:

In [17]:
%run --values 4 5
parameter: values = list

print(values)

INFO: Running default :

['4', '5']


INFO: Workflow default (ID=27fc2a7435e0338c) is executed successfully with 1 completed step.

### SoS types

SoS types such as `file_target` and `sos_targets`. Generally speaking you can pass filenames as `str` or list of `str`, but passing SoS types such as `file_target` allow you to create variable in these types without type coercion.

For example, the `file_target` type is derived from [`pathlib.Path`](https://docs.python.org/3.6/library/pathlib.html) with automatic expansion of `~` and other features. If you pass a parameter with type `file_target`, SoS will convert passed string into a `path` object so that you can use it directly.

In [18]:
%run --infile ~/project/a.txt
parameter: infile = file_target

print(infile.name)
print(infile.parent)
print(infile.exists())

INFO: Running default :

a.txt
/Users/bpeng1/project
False


INFO: Workflow default (ID=9cb95e36162ffbe0) is executed successfully with 1 completed step.

Similarly, if you pass a `paths` (a sequence of `path`), you parameter will be of type `paths`:

In [19]:
%run --infiles a.txt b.txt
parameter: infiles = paths('a.txt')

print(infiles)

INFO: Running default :

a.txt b.txt


INFO: Workflow default (ID=f0ba86b384fdc90a) is executed successfully with 1 completed step.

## Scope of parameters

Parameters are usually defined in the `global` section of workflows, but it can also be defined in steps, in which case the scope of the parameter is limited to the step in which it is defined.

The following example has `var_A` and `var_B` defined in steps `A` and `B`. Although `var_B` is specified with a type, it is not required when workflow `A` is executed.

In [20]:
%run A --var_A 10

[A]
parameter: var_A = int
print(f'var A is {var_A}')

[B]
parameter: var_B = int
print(f'var B is {var_B}')

INFO: Running A :

var A is 10


INFO: Workflow A (ID=736395fed491ea67) is executed successfully with 1 completed step.

## Parameters and subworkflows

Parameters are usually specified from command line but it can be specified from a subworkflow as it the (nested) workflow is executed with specified parameter.

In [21]:
%run

[sub]
parameter: val = 10
print(f'Calling workflow sub with parameter {val}')

[default]
sos_run('sub', val=5)
sos_run('sub', val=15)

INFO: Running default :

INFO: Running sub :

Calling workflow sub with parameter 5


INFO: Running sub :

Calling workflow sub with parameter 15


INFO: Workflow default (ID=e974085279b5c897) is executed successfully with 3 completed steps.

The `%run` magic in this example executes the default workflow `default`, and executes the `sub` subworkflow with different parameters. You can also specify the workflow to execute directly.

In [22]:
%run sub --val 20

[sub]
parameter: val = 10
print(f'Calling workflow sub with parameter {val}')

[default]
sos_run('sub', val=5)
sos_run('sub', val=15)

INFO: Running sub :

Calling workflow sub with parameter 20


INFO: Workflow sub (ID=e3f74bdb91343a41) is executed successfully with 1 completed step.

## Further reading <a id='further_reading'></a>

* [SoS Data Types](doc/user_guide/sos_datatypes.html)